In [6]:
from pathlib import Path
import os

def find_project_root(max_up=8):
    """Walk up until we see a Vite/Cesium project root (public/ + package.json)."""
    p = Path.cwd().resolve()
    for _ in range(max_up+1):
        if (p / "public").exists() and (p / "package.json").exists():
            return p
        p = p.parent
    raise FileNotFoundError(f"Could not find project root from {Path.cwd()}")

ROOT_DIR = find_project_root()
DATA_DIR = ROOT_DIR / "public" / "data"
OUT_DIR  = ROOT_DIR / "analysis" / "outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("ROOT_DIR:", ROOT_DIR)
print("DATA_DIR:", DATA_DIR)
print("OUT_DIR :", OUT_DIR)

# Sanity: list data files you’ll use
for name in ["g1_air_quality_animated.json", "zone_traffic_aggregated.json"]:
    fp = DATA_DIR / name
    print(name, "OK" if fp.exists() else "MISSING", "-", fp)


ROOT_DIR: C:\glasgow-carbon-map\Main_project
DATA_DIR: C:\glasgow-carbon-map\Main_project\public\data
OUT_DIR : C:\glasgow-carbon-map\Main_project\analysis\outputs
g1_air_quality_animated.json OK - C:\glasgow-carbon-map\Main_project\public\data\g1_air_quality_animated.json
zone_traffic_aggregated.json OK - C:\glasgow-carbon-map\Main_project\public\data\zone_traffic_aggregated.json


In [7]:
import json, pandas as pd, numpy as np

# Analysis window (G1 only)
START = pd.Timestamp("2023-05-08 00:00:00")
END   = pd.Timestamp("2023-07-12 10:00:00")

aq_path = DATA_DIR / "g1_air_quality_animated.json"
tr_path = DATA_DIR / "zone_traffic_aggregated.json"

# Air quality (G1)
aq = pd.DataFrame.from_dict(json.loads(aq_path.read_text()), orient="index")
aq.index = pd.to_datetime(aq.index)
aq = aq.sort_index().loc[START:END]

# Traffic (keep G1 column)
tr_all = pd.DataFrame.from_dict(json.loads(tr_path.read_text()), orient="index")
tr_all.index = pd.to_datetime(tr_all.index)
tr_all = tr_all.sort_index().loc[START:END]
tr_g1 = tr_all["G1"].astype(float)

print("Hours in window:", len(pd.date_range(START, END, freq="H")))
print("AQ coverage:", aq.notna().mean().round(3).to_dict())
print("Traffic G1 coverage:", float(tr_g1.notna().mean().round(3)))


Hours in window: 1571
AQ coverage: {'no2': 1.0, 'pm25': 0.968, 'o3': 0.992, 'pm10': 0.969}
Traffic G1 coverage: 1.0


C:\Users\Prajw\AppData\Local\Temp\ipykernel_28884\453220425.py:21: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  print("Hours in window:", len(pd.date_range(START, END, freq="H")))


In [8]:
MAX_LAG = 24
MIN_N   = 12

def pearson_r(x, y):
    x = np.asarray(x); y = np.asarray(y)
    if len(x) < 2: return np.nan, len(x)
    xc, yc = x - x.mean(), y - y.mean()
    denom = np.sqrt((xc**2).sum() * (yc**2).sum())
    if denom == 0: return np.nan, len(x)
    return float((xc * yc).sum() / denom), len(x)

def r_ci(r, n):
    if not np.isfinite(r) or n < 4: return (np.nan, np.nan)
    z = np.arctanh(r); se = 1/np.sqrt(n-3)
    return float(np.tanh(z - 1.96*se)), float(np.tanh(z + 1.96*se))

results, lag_curves = [], {}

for metric in ["no2","pm25","pm10","o3"]:
    s_aq = aq[metric].astype(float)
    s_tr = tr_g1

    rows = []
    for lag in range(-MAX_LAG, MAX_LAG+1):
        # positive lag: AQ lags traffic
        A = s_aq.shift(lag) if lag > 0 else s_aq
        T = s_tr if lag > 0 else s_tr.shift(-lag)  # AQ leads when lag<0
        df = pd.concat([A, T], axis=1, keys=["aq","tr"]).dropna()
        if len(df) >= MIN_N:
            r, n = pearson_r(df["aq"].values, df["tr"].values)
        else:
            r, n = (np.nan, len(df))
        rows.append({"lag_h": lag, "r": r, "n": n})

    lc = pd.DataFrame(rows)
    lag_curves[metric] = lc

    best = lc.loc[lc["r"].abs().idxmax()]
    r_best, lag_best, n_best = float(best.r), int(best.lag_h), int(best.n)
    lo, hi = r_ci(r_best, n_best)

    results.append({
        "zone": "G1",
        "metric": metric.upper(),
        "best_r": round(r_best,3),
        "best_lag_h": lag_best,
        "direction": "AQ lags" if lag_best>0 else ("AQ leads" if lag_best<0 else "simult."),
        "n": n_best,
        "ci_low": round(lo,3),
        "ci_high": round(hi,3)
    })

res = pd.DataFrame(results)
res.to_csv(OUT_DIR / "g1_corr_summary.csv", index=False)
res


,zone,metric,best_r,best_lag_h,direction,n,ci_low,ci_high
0,G1,NO2,0.190,-14,AQ leads,939,0.128,0.251
1,G1,PM25,-0.049,-9,AQ leads,908,-0.114,0.016
2,G1,PM10,0.048,20,AQ lags,897,-0.017,0.114
3,G1,O3,-0.398,-18,AQ leads,931,-0.450,-0.342


In [9]:
import matplotlib.pyplot as plt

for m, lc in lag_curves.items():
    plt.figure(figsize=(6,3))
    plt.axhline(0, lw=1, color="k", alpha=.35)
    plt.plot(lc["lag_h"], lc["r"], marker="o", ms=3)
    plt.title(f"G1 – {m.upper()} vs Traffic (lag scan ±24 h)")
    plt.xlabel("Lag (hours)  [positive = AQ lags]")
    plt.ylabel("Pearson r")
    plt.tight_layout()
    plt.savefig(OUT_DIR / f"g1_{m}_lag_scan.png", dpi=160)
    plt.close()

print("Saved:", OUT_DIR / "g1_corr_summary.csv")
print("and lag–r PNGs:", list((OUT_DIR).glob("g1_*_lag_scan.png")))


Saved: C:\glasgow-carbon-map\Main_project\analysis\outputs\g1_corr_summary.csv
and lag–r PNGs: [WindowsPath('C:/glasgow-carbon-map/Main_project/analysis/outputs/g1_no2_lag_scan.png'), WindowsPath('C:/glasgow-carbon-map/Main_project/analysis/outputs/g1_o3_lag_scan.png'), WindowsPath('C:/glasgow-carbon-map/Main_project/analysis/outputs/g1_pm10_lag_scan.png'), WindowsPath('C:/glasgow-carbon-map/Main_project/analysis/outputs/g1_pm25_lag_scan.png')]


In [10]:
# Baseline (mirror your app defaults; adjust if your panel uses different values)
shares = dict(car=0.78, lgv=0.14, hgv=0.05, bus=0.03)   # auto-normalised
ef     = dict(car=0.18, lgv=0.25, hgv=0.85, bus=0.82)   # kg CO2 per km
d_g1   = 0.5  # km per vehicle in G1

def eff_ef(sh, ef):
    s = sum(sh.values()); shn = {k:v/s for k,v in sh.items()}
    return sum(shn[k]*ef[k] for k in shn)

EFeff = eff_ef(shares, ef)
co2_hourly = tr_g1 * EFeff * d_g1  # kg CO2/h
co2_hourly.describe()


count    1545.000000
mean      108.746888
std        59.700745
min         8.002500
25%        51.652500
50%       115.308750
75%       159.080000
max       262.385000
Name: G1, dtype: float64

In [11]:
scenarios = [
    ("Baseline", shares, d_g1),
    ("HGV +5pp", {**shares, "hgv": shares["hgv"]+0.05, "car": shares["car"]-0.05}, d_g1),
    ("HGV -3pp", {**shares, "hgv": max(0, shares["hgv"]-0.03), "car": shares["car"]+0.03}, d_g1),
    ("d +0.2 km", shares, d_g1+0.2),
    ("d -0.2 km", shares, max(0.1, d_g1-0.2)),
]

bars=[]
for name,sh,d in scenarios:
    total_tonnes = float((tr_g1 * eff_ef(sh,ef) * d).sum())/1000.0
    bars.append((name, total_tonnes))

labels, tonnes = zip(*bars)
plt.figure(figsize=(6,3))
plt.bar(labels, tonnes)
plt.ylabel("Total CO₂ (tonnes) – G1, 2023-05-08→07-12")
plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig(OUT_DIR / "g1_emissions_sensitivity.png", dpi=160)
plt.close()

print("Saved:", OUT_DIR / "g1_emissions_sensitivity.png")


Saved: C:\glasgow-carbon-map\Main_project\analysis\outputs\g1_emissions_sensitivity.png
